In [2]:
%use dataframe
%useLatestDescriptors

In [1]:
import org.springframework.web.reactive.function.client.WebClient
import org.springframework.web.reactive.function.client.bodyToFlow
import org.springframework.web.reactive.function.client.exchangeToFlow
import dev.example.ChatInput
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.Flow
import org.springframework.web.reactive.function.client.*

In [3]:
val exchangeStrategies = ExchangeStrategies.builder()
    .codecs { configurer ->
        configurer.defaultCodecs().maxInMemorySize(10 * 1024 * 1024) // 10 MB
    }
    .build()

val webClient = WebClient.builder().exchangeStrategies(exchangeStrategies).build()

In [4]:
fun WebClient.streamingChat(chatRequest: ChatInput): Flow<String> {
    val url = "http://localhost:8080/ai/stream"
    return this.post()
        .uri(url)
        .bodyValue(chatRequest)
        .retrieve()
        .bodyToFlow<String>()
}

In [6]:
import kotlinx.coroutines.runBlocking

runBlocking {

    val prompt = ChatInput("""Tell me a joke about cats that is max 50 words long""")

    webClient.streamingChat(prompt).collect {
        println(it) // prints the response line by line

    }
}

What do you call a pile of cats? 

A meow-tain!


In [5]:

fun WebClient.chat(chatRequest: ChatInput): Flow<String> {
    val url = "http://localhost:8080/ai/chat"
    return this.post()
        .uri(url)
        .bodyValue(chatRequest)
        .retrieve()
        .bodyToFlow<String>()
}


In [6]:
var history:String = ""
fun chat(message: String, messageId:String) = runBlocking {
        print(history + "\n\n")
        webClient.streamingChat(ChatInput(message, messageId)).collect {
        println(it)// prints the response line by line
            history += it

        }
    }


In [17]:
val conversationId = "12345633"

In [18]:
weather("What's the weather in Amsterdam in Celcius?" , conversationId)
//chat("What do you want?" , conversationId)
//chat("As a starter I want something very cheesy with mozarella, tomatoes and Basilicum" , conversationId)
//chat("I want something cheezy with mozarella, pasta and meat" , conversationId)

//chat("I want the mozarella dish" , conversationId)
//chat("I want to go with that" , conversationId)
//chat("I want a pizza with meat, meatballs and various cheeses" , conversationId)
//chat("Spaghetti Carbonara" , conversationId)
//chat("As dessert I want  something with a lot of cream, berries and caramel" , conversationId)
//chat("Cream Tart" , conversationId)
//chat("The order is complete" , conversationId)

The current temperature in Amsterdam is 30°C.


In [98]:
import org.springframework.web.reactive.function.client.awaitBody
import dev.example.*
import java.io.ByteArrayInputStream

suspend fun  WebClient.speech(chatRequest: ChatInput): ByteArray {
    val url = "http://localhost:8080/ai/speech"
    return this.post()
        .uri(url)
        .bodyValue(chatRequest)
        .retrieve()
        .awaitBody<ByteArray>()
}
fun speech(message: String, messageId:String) = runBlocking {
    playMp3(ByteArrayInputStream(webClient.speech(ChatInput(message, messageId))))
}
val conversationId = "12345632"

In [101]:
//speech("What's up?" , conversationId)
//speech("As main dish I want a pizza with meat, meatballs and various cheeses" , conversationId)
speech("As dessert I want  something with a lot of cream, berries and caramel" , conversationId)